http://shop.oreilly.com/product/0636920232582.do 

First import the necessary stuff:

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import (
ensemble,
preprocessing,
tree,)

In [2]:
from sklearn.metrics import (
auc,
confusion_matrix,
roc_auc_score,
roc_curve,
)

In [3]:
from sklearn.model_selection import (
train_test_split,
StratifiedKFold,
)

In [4]:
from yellowbrick.classifier import (
ConfusionMatrix,
ROCAUC,
)

ModuleNotFoundError: No module named 'yellowbrick'

In [5]:
from yellowbrick.model_selection import (
LearningCurve,
)

Now load the data

In [6]:
url=("http://biostat.mc.vanderbilt.edu/"
     "wiki/pub/Main/DataSets/titanic3.xls"
    )

df = pd.read_excel(url)
orig_df=df

Analyze the data

In [7]:
df.dtypes

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

In [8]:
import pandas_profiling as pp
pp.ProfileReport(df)

In [9]:
df.shape

(1309, 14)

In [10]:
df.describe().iloc[:,:2]

,pclass,survived
count,1309.000000,1309.000000
mean,2.294882,0.381971
std,0.837836,0.486055
min,1.000000,0.000000
25%,2.000000,0.000000
50%,3.000000,0.000000
75%,3.000000,1.000000
max,3.000000,1.000000


In [11]:
df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

df.isnull().mean()

In [12]:
df.isnull().sum(axis=1).loc[:10]

0     1
1     1
2     2
3     1
4     2
5     1
6     1
7     2
8     1
9     2
10    1
dtype: int64

In [13]:
mask=df.isnull().any(axis=1)

In [14]:
mask.head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [15]:
df[mask].body.head()

0      NaN
1      NaN
2      NaN
3    135.0
4      NaN
Name: body, dtype: float64

In [16]:
df.sex.value_counts(dropna=False)

male      843
female    466
Name: sex, dtype: int64

In [17]:
df.embarked.value_counts(dropna=False)

S      914
C      270
Q      123
NaN      2
Name: embarked, dtype: int64

Now we Create the Features

In [18]:
name=df.name

In [19]:
name.head(3)

0     Allen, Miss. Elisabeth Walton
1    Allison, Master. Hudson Trevor
2      Allison, Miss. Helen Loraine
Name: name, dtype: object

In [20]:
df=df.drop(
columns=[
    "name",
    "ticket",
    "home.dest",
    "boat",
    "body",
    "cabin",
])

In [21]:
df=pd.get_dummies(df)
df.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_female',
       'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [22]:
df=df.drop(columns="sex_male")

In [23]:
df.columns

Index(['pclass', 'survived', 'age', 'sibsp', 'parch', 'fare', 'sex_female',
       'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [24]:
y=df.survived
X=df.drop(columns="survived")

Sample Data p.25

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.3, random_state=42)

In [26]:
df.isnull().sum()

pclass          0
survived        0
age           263
sibsp           0
parch           0
fare            1
sex_female      0
embarked_C      0
embarked_Q      0
embarked_S      0
dtype: int64

In [31]:
meds = X_train.median()
X_train = X_train.fillna(meds)
X_test = X_test.fillna(meds)

Baseline Model p.29

In [35]:
from sklearn.dummy import DummyClassifier
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test) # Accuracy

0.4732824427480916

In [36]:
from sklearn import metrics
metrics.precision_score(
y_test, bm.predict(X_test)
)

0.42038216560509556